In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [24]:
import os
from time import time

import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt
import pandas as pd

## Mixed Plotting Experiment - Temperature Data (Choropleth) with Power Plant Data (Scatter) and CO2 Data (Bubble Plot)

### Load/Preprocess Data

#### Power Plant

In [3]:
t0 = time()
power_df = pd.read_csv('https://docs.google.com/spreadsheets/d/1RkEs8QgM9aeRU4LJsx1D587tpgsLgpHbqdtQlQH-wOA/export?format=csv', index_col=0)
print('Load time: %i sec' % (time() - t0))

Load time: 17 sec


In [4]:
# Plotting constants 
MIN_MARKER_SIZE = 4
MAX_MARKER_SIZE = 20
START_YEAR = 1970
END_YEAR = 1980

In [5]:
def get_formatting(df, min_point, max_point, min_netgen, max_netgen):
    
    # Get Colors for points
    if df['Class'] == 'Clean':
        vals = ['rgb(0, 128, 0)','rgb(144, 238, 144)']
    elif df['Class'] == 'Fossil':
        vals = ['rgb(255, 0, 0)','rgb(242, 177, 172)']
    else:
        raise Exception('Invalid Class {} found.'.format(Class))
        
    #Normalize netgen data to 0-1 range
    vals.append(min_point + ((df['netgen'] - min_netgen) / (max_netgen - min_netgen)) * (max_point - min_point))
    return vals

max_netgen = power_df['netgen'].max()
min_netgen = power_df['netgen'].min()
power_df['Formatting'] = power_df.apply(lambda x: get_formatting(x, MIN_MARKER_SIZE, MAX_MARKER_SIZE, min_netgen, max_netgen), axis=1)

#### Temperature

In [6]:
from load_temperature_data import get_all_states

t0 = time()
temp_df = get_all_states()
print('Load time: %i sec' % (time() - t0))

Load time: 49 sec


#### CO2

In [7]:
co2_df = pd.read_csv(os.path.join('..', 'data', 'CO2-data.csv'))

In [8]:
# Set longitude to (-180, +180]
co2_df['DLONG'] = co2_df['LONG'] - 180
# Set longitude to [-90, +90)
co2_df['DLAT'] = -co2_df['LAT'] + 90

In [9]:
# Filter by latlon to narrow geographic area
co2_USA = co2_df[(co2_df["DLONG"] < -60) & (co2_df["DLAT"] > 15)]

### Single year plot

In [53]:
power_df_1980 = power_df[power_df['year'] == 1980]
temp_df_1980 = temp_df.xs(1980, level='year')
MIN_CO2 = 10
co2_df_1980 = co2_USA[co2_USA['1980'] > 10]

In [82]:
data = [
    go.Choropleth(
        name='Temperature',
        locations=temp_df_1980.reset_index()['state'],
        text=temp_df_1980['10y'].apply(lambda x: '%.2f °C' % x),
        hoverinfo='location+text+name',
        z=temp_df_1980['10y'],
        zmin=temp_df.min()['10y'],
        zmid=0,
        zmax=temp_df.max()['10y'],
        locationmode='USA-states',
        colorscale='RdBu',
        colorbar=dict(
            title='Temperature',
            x=-0.1,
            xanchor='left',
            ticksuffix=' ° C',
            showticksuffix='last',
            len=.6
        )
    ),
    go.Scattergeo(
        name='CO<sub>2</sub>',
        lat=co2_df_1980['DLAT'],
        lon=co2_df_1980['DLONG'],
        text=co2_df_1980['1980'].apply(lambda x: '%.2f KG' % x),
        marker=go.scattergeo.Marker(
            opacity=0.7,
            size=co2_df_1980['1980']/1.5,
            symbol='circle',
            line=dict(
                width=0
            ),
            color=co2_df_1980['1980'],
            cmin=MIN_CO2,
            cmax=co2_df_1980['1980'].max(),
            colorscale=[
                (0, '#969696'),
                (0.333, '#737373'),
                (0.667, '#525252'),
                (1, '#252525')
            ],
            colorbar=dict(
                title='CO<sub>2</sub>',
                xanchor='left',
                ticksuffix=' KG',
                showticksuffix='last',
                len=.6
            ) 
        )
    ),
    go.Scattergeo(
        name='Power',
        locationmode='USA-states',
        lat=power_df_1980['Latitude'],
        lon=power_df_1980['Longitude'],
        mode='markers',
        marker=go.scattergeo.Marker(
            size=[i[2] for i in power_df_1980['Formatting'].tolist()],
            color=[i[0] for i in power_df_1980['Formatting'].tolist()],
            opacity=0.7,
            line=dict(
                width=0.5
            )
        ),
        text = power_df_1980[['plant name','netgen']].apply(
            lambda j: '<br>Net Generation (MWh): '.join(j.astype(str)),
            axis=1
        ),
        hoverinfo='text+name'
    )
]

layout = go.Layout(
    title = go.layout.Title(
        text = '1980 Power Plant, Temperature, and CO2 Data'
    ),
    width=1000,
    height=750,
    autosize=False,
    showlegend=False,
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'),
    updatemenus=[
        dict(
            type = 'buttons',
            active=0,
            buttons=[
                dict(
                    args=[{'visible': [True, True, True]}],
                    label='All',
                    method='update'
                ),
                dict(
                    args=[{'visible': [True, False, False]}],
                    label='Temperature',
                    method='update'
                ),
                dict(
                    args=[{'visible': [False, True, False]}],
                    label='CO2',
                    method='update'
                ),
                dict(
                    args=[{'visible': [False, False, True]}],
                    label='Power',
                    method='update'
                )
            ],
            direction = 'left',
            pad = {'r': 10, 't': 10},
            x = 0.1,
            xanchor = 'left',
            y = 1.1,
            yanchor = 'top' 
        ),
    ]
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'd3-combo-map')

### Multi-year with Slider

In [104]:
MIN_TEMP = temp_df.min()['10y']
MAX_TEMP = temp_df.max()['10y']

In [105]:
def get_temp_trace_for_year(year):
    temp_df_yr = temp_df.xs(year, level='year')
    return go.Choropleth(
        visible=False,
        name='Temperature',
        locations=temp_df_yr.reset_index()['state'],
        text=temp_df_yr['10y'].apply(lambda x: '%.2f °C' % x),
        hoverinfo='location+text+name',
        z=temp_df_yr['10y'],
        zmin=MIN_TEMP,
        zmid=0,
        zmax=MAX_TEMP,
        locationmode='USA-states',
        colorscale='RdBu',
        colorbar=dict(
            title='Temperature',
            x=-0.1,
            xanchor='left',
            ticksuffix=' ° C',
            showticksuffix='last'
        )
    )

In [106]:
MIN_CO2 = 10 # To reduce plot noise
MAX_CO2 = co2_USA.iloc[:, 3:-2].max().max()

print(MAX_CO2)

60.152330000000006


In [107]:
def get_co2_trace_for_year(year):
    co2_df_yr = co2_USA[co2_USA[str(year)] > MIN_CO2]
    return go.Scattergeo(
        visible=False,
        name='CO<sub>2</sub>',
        lat=co2_df_yr['DLAT'],
        lon=co2_df_yr['DLONG'],
        text=co2_df_yr[str(year)].apply(lambda x: '%.2f KG' % x),
        marker=go.scattergeo.Marker(
            opacity=0.7,
            size=co2_df_yr[str(year)]/1.5,
            symbol='circle',
            line=dict(
                width=0
            ),
            color=co2_df_yr[str(year)],
            cmin=MIN_CO2,
            cmax=MAX_CO2,
            colorscale=[
                (0, '#969696'),
                (0.333, '#737373'),
                (0.667, '#525252'),
                (1, '#252525')
            ],
            colorbar=dict(
                title='CO<sub>2</sub>',
                xanchor='left'
            ) 
        )
    )

In [108]:
def get_power_trace_for_year(year):
    power_df_yr = power_df[power_df['year'] == year]
    return go.Scattergeo(
        visible=False,
        name='Power',
        locationmode='USA-states',
        lat=power_df_yr['Latitude'],
        lon=power_df_yr['Longitude'],
        mode='markers',
        marker=go.scattergeo.Marker(
            size=[i[2] for i in power_df_yr['Formatting'].tolist()],
            color=[i[0] for i in power_df_yr['Formatting'].tolist()],
            opacity=0.7,
            line=dict(
                width=0.5
            )
        ),
        text = power_df_yr[['plant name','netgen']].apply(
            lambda j: '<br>Net Generation (MWh): '.join(j.astype(str)),
            axis=1
        ),
        hoverinfo='text+name'
    )

In [118]:
start_year = 1970
end_year = 2000
year_count = len(range(start_year, end_year + 1))
trace_count = 3
title='Power Plant, Temperature, and CO2 Data'

In [119]:
data = []
for y in range(start_year, end_year + 1):
    data.extend(
        [
            get_temp_trace_for_year(y),
            get_co2_trace_for_year(y),
            get_power_trace_for_year(y)
        ]
    )

In [120]:
for i, _ in enumerate(data[:trace_count]):
    data[i]['visible'] = True

In [121]:
def display_nth_year(n):
    visibility = []
    for y in range(year_count):
        if y == n:
            visibility.extend([True] * trace_count)
        else:
            visibility.extend([False] * trace_count)
    return visibility

In [122]:
steps = []
for i in range(start_year, end_year + 1):
    step = dict(
        method = 'update',
        label = str(i),
        args = [
            {'visible': display_nth_year(i - start_year)},
            {'title.text': title + ' (%d)' % i}],
    )
    steps.append(step)

slider = [
    dict(
        active = start_year,
        currentvalue = {"prefix": "Year: "},
        activebgcolor = '#d9bec8',
        pad = {"t": 50},
        steps = steps
    )
]

In [123]:
layout = go.Layout(
    title = go.layout.Title(
        text = title + ' (%d)' % start_year
    ),
    width=1000,
    height=750,
    autosize=False,
    showlegend=False,
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'
    ),
    sliders=slider
)

In [124]:
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'd3-combo-slider-map')